In [9]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
import hashlib

# ==========================================
# 题目数据
# ==========================================
a = [965032030645819473226880279, 699680391768891665598556373, 1022177754214744901247677527, 680767714574395595448529297, 1051144590442830830160656147, 1168660688736302219798380151, 796387349856554292443995049, 740579849809188939723024937, 940772121362440582976978071, 787438752754751885229607747, 1057710371763143522769262019, 792170184324681833710987771, 912844392679297386754386581, 906787506373115208506221831, 1073356067972226734803331711, 1230248891920689478236428803, 713426848479513005774497331, 979527247256538239116435051, 979496765566798546828265437, 836939515442243300252499479, 1185281999050646451167583269, 673490198827213717568519179, 776378201435505605316348517, 809920773352200236442451667, 1032450692535471534282750757, 1116346000400545215913754039, 1147788846283552769049123803, 994439464049503065517009393, 825645323767262265006257537, 1076742721724413264636318241, 731782018659142904179016783, 656162889354758353371699131, 1045520414263498704019552571, 1213714972395170583781976983, 949950729999198576080781001, 1150032993579134750099465519, 975992662970919388672800773, 1129148699796142943831843099, 898871798141537568624106939, 997718314505250470787513281, 631543452089232890507925619, 831335899173370929279633943, 1186748765521175593031174791, 884252194903912680865071301, 1016020417916761281986717467, 896205582917201847609656147, 959440423632738884107086307, 993368100536690520995612807, 702602277993849887546504851, 1102807438605649402749034481, 629539427333081638691538089, 887663258680338594196147387, 1001965883259152684661493409, 1043811683483962480162133633, 938713759383186904819771339, 1023699641268310599371568653, 784025822858960757703945309, 986182634512707587971047731, 1064739425741411525721437119, 1209428051066908071290286953, 667510673843333963641751177, 642828919542760339851273551, 1086628537309368288204342599, 1084848944960506663668298859, 667827295200373631038775959, 752634137348312783761723507, 707994297795744761368888949, 747998982630688589828284363, 710184791175333909291593189, 651183930154725716807946709, 724836607223400074343868079, 1118993538091590299721647899]
b = 34962396275078207988771864327
ciphertext = b'Lo~G\xf46>\xd609\x8e\x8e\xf5\xf83\xb5\xf0\x8f\x9f6&\xea\x02\xfa\xb1_L\x85\x93\x93\xf7,`|\xc6\xbe\x05&\x85\x8bC\xcd\xe6?TV4q'

def solve():
    n = len(a)
    print(f"[*] Problem dimension: n = {n}")
    
    # 使用极其巨大的权重 K，彻底消除部分和的局部极小值干扰
    K = 2**200 
    
    # 构造精确的 (n+1) x (n+1) 矩阵
    M = Matrix(ZZ, n + 1, n + 1)
    
    for i in range(n):
        M[i, i] = 2          # 对角线为 2
        M[i, n] = K * a[i]   # 最后一列乘以巨大权重 K
        
    for i in range(n):
        M[n, i] = 1          # 最后一行前 n 列为 1
    M[n, n] = K * b          # 右下角为 K * b
    
    print("[*] Reconstructed highly weighted CJLOSS lattice.")
    
    # 检查矩阵并提取 p 的函数
    def check_matrix(B_mat):
        for row in B_mat:
            # 严格检查最后一列是否为0（由于权重K极大，这里为0意味着100%满足背包和）
            if row[n] != 0:
                continue
                
            vec = row[:n]
            # 检查前n项是否全为 1 或 -1
            if all(abs(v) == 1 for v in vec):
                print(f"[+] Found short vector (1/-1): {vec[:10]}...")
                
                # 格基规约可能找到目标向量，也可能找到它的相反数 (-vec)
                # 因此我们生成两种情况进行测试
                x_candidates = [
                    [(v + 1) // 2 for v in vec],
                    [(-v + 1) // 2 for v in vec]
                ]
                
                for x in x_candidates:
                    calc_sum = sum(x[i] * a[i] for i in range(n))
                    if calc_sum == b:
                        print(f"[+] Subset sum verified successfully!")
                        
                        # 还原 p
                        p = 0
                        for i in range(n):
                            p += int(x[i]) * (1 << i)
                            
                        print(f"[*] Recovered p: {p}")
                        
                        # 解密
                        key = hashlib.sha256(str(p).encode()).digest()
                        cipher = AES.new(key, AES.MODE_ECB)
                        try:
                            pt = cipher.decrypt(ciphertext)
                            flag = unpad(pt, 16).decode()
                            print(f"[SUCCESS] FLAG: {flag}")
                            return True
                        except Exception as e:
                            print(f"[-] Padding error during decryption: {e}")
        return False

    block_size  = 30
    B_bkz = M.BKZ(block_size=block_size)
    if check_matrix(B_bkz):
        return
            
    print("\n[-] All lattice reduction techniques failed. Please check the inputs.")

if __name__ == "__main__":
    solve()

[*] Problem dimension: n = 72
[*] Reconstructed highly weighted CJLOSS lattice.
[+] Found short vector (1/-1): (-1, -1, 1, -1, -1, 1, -1, 1, -1, 1)...
[+] Subset sum verified successfully!
[*] Recovered p: 4208626653103825685156
[SUCCESS] FLAG: LILCTF{M4ybe_7he_brut3_f0rce_1s_be5t}
